clean version available at:  
- https://github.com/Personalization-Technologies-Lab/RecSys-Course-HSE-Fall23/tree/main/Seminar5

Installing packages:
```
# polara
pip install --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara
```

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme(style='white', context='paper')
%config InlineBackend.figure_format = "svg"

from polara import get_movielens_data
from polara.evaluation.pipelines import find_optimal_config, random_grid

from dataprep import transform_indices, leave_last_out, verify_time_split, reindex_data
from evaluation import topn_recommendations, model_evaluate, downvote_seen_items

# Prepraring data

In [3]:
data = get_movielens_data(include_time=True)

In [4]:
training_, holdout_ = leave_last_out(data, 'userid', 'timestamp')
verify_time_split(training_, holdout_)

In [5]:
training, data_index = transform_indices(training_, 'userid', 'movieid')
holdout = (
    reindex_data(holdout_, data_index, filter_invalid=True)
    .sort_values('userid')
)

In [6]:
data_description = dict(
    users = data_index['users'].name,
    items = data_index['items'].name,
    feedback = 'rating',
    n_users = len(data_index['users']),
    n_items = len(data_index['items']),
    test_users = holdout[data_index['users'].name].values,
)

In [7]:
userid = data_description['users']
seen_idx_mask = training[userid].isin(data_description['test_users'])
testset = training[seen_idx_mask]

# SGD MF

$$
\begin{gather}
e_{ui} =  ... \\
\mathbb{p}_u\gets\mathbb{p}_u+\eta\left(e_{ui}\mathbb{q}_i-\lambda\mathbb{p}_u\right)\\
\mathbb{q}_i\gets\mathbb{q}_i+\eta\left(e_{ui}\mathbb{p}_u-\lambda\mathbb{q}_i\right)
\end{gather}
$$


In [8]:
def mf_sgd_build(config, data, data_description):
    ... # implement MF training here
    return sgd_params


In [9]:
config = {
    "rank": 30,
    "n_epochs": 1,
    "learning_rate": 1e-3,
    "regularization": 1,
    "seed": 20230711
}

In [12]:
sgd_params = mf_sgd_build(config, training, data_description)

In [11]:
def mf_sgd_scoring(params, data, data_description):
    ... # implement the scoring with MF
    return scores

In [45]:
sgd_scores = mf_sgd_scoring(sgd_params, None, data_description)
downvote_seen_items(sgd_scores, testset, data_description)

In [ ]:
sgd_recs = topn_recommendations(sgd_scores, topn=10)
model_evaluate(sgd_recs, holdout, data_description)